In [2]:
from datetime import datetime
# importing moules
import os
import pandas as pd
import matplotlib
import numpy as np
from re import search
import matplotlib.pyplot as plt
from datetime import date
import seaborn as sns
import math

In [3]:
all_scz_diagnoses = pd.read_csv('/Users/valena17/psych_nlp/data/clean_ccsr_scz_patients.csv')

/var/folders/xm/m4dy5bvs1fn7fb9wxjhsg4dc0000gn/T/ipykernel_8430/795740903.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  all_scz_diagnoses = pd.read_csv('/Users/valena17/psych_nlp/data/clean_ccsr_scz_patients.csv')


In [4]:
codes_scz = set(all_scz_diagnoses[all_scz_diagnoses['CCSR_wSubstance'] == 'Schizophrenia spectrum and other psychotic disorders'].Value)
codes_depression = set(all_scz_diagnoses[all_scz_diagnoses['CCSR_wSubstance'] == 'Depressive disorders'].Value)
codes_trauma = set(all_scz_diagnoses[all_scz_diagnoses['CCSR_wSubstance'] == 'Trauma- and stressor-related disorders'].Value)
codes_bipolar = set(all_scz_diagnoses[all_scz_diagnoses['CCSR_wSubstance'] == 'Bipolar and related disorders'].Value)
codes_anxiety = set(all_scz_diagnoses[all_scz_diagnoses['CCSR_wSubstance'] == 'Anxiety and fear-related disorders'].Value)
codes_substance = set(all_scz_diagnoses[all_scz_diagnoses['CCSR_wSubstance'] == 'Substance-related disorders'].Value)

In [5]:
print(len(codes_scz), len(codes_depression), len(codes_trauma), len(codes_bipolar), len(codes_anxiety),
      len(codes_substance))
print(codes_scz, "\n", codes_depression, "\n", codes_trauma, "\n", codes_bipolar, "\n", codes_anxiety, "\n", codes_substance)

22 20 23 34 20 207
{'F20.5', 'F28', 'F21', 'F06.1', 'F20.0', 'F25.0', 'F25.1', 'F25', 'F25.9', 'F20.81', 'F06.0', 'F06.2', 'F20.1', 'F20.9', 'F20.2', 'F20.3', 'F20.89', 'F23', 'F24', 'F22', 'F25.8', 'F29'} 
 {'F32.1', 'F32.89', 'F32.2', 'F33.1', 'F33.3', 'F32.81', 'F32.8', 'F34.1', 'F32.0', 'F33.9', 'F32.3', 'F33.41', 'F33.8', 'F33.0', 'F32.4', 'F32.A', 'F33.2', 'F06.32', 'F06.31', 'F32.9'} 
 {'F44.9', 'F43.8', 'F43.81', 'F44.4', 'F44.89', 'F43.9', 'F43.25', 'F44.6', 'F43.12', 'F48.1', 'F43.10', 'F43.22', 'F43.23', 'F43.11', 'F43.89', 'F43.0', 'F43.20', 'F43.29', 'F44.1', 'F44.5', 'F44.81', 'F43.21', 'F43.24'} 
 {'F31.73', 'F30.3', 'F30.8', 'F31.4', 'F34.0', 'F31.5', 'F06.33', 'F31.13', 'F31.30', 'F30.13', 'F30.10', 'F30.9', 'F31.61', 'F31.10', 'F31.31', 'F31.0', 'F31.81', 'F31.2', 'F31.77', 'F30.12', 'F31.12', 'F31.11', 'F31.75', 'F31.9', 'F30.2', 'F06.34', 'F30.11', 'F31.62', 'F31.60', 'F31.71', 'F31.32', 'F31.89', 'F31.63', 'F31.64'} 
 {'F40.240', 'F41.1', 'F41.8', 'F40.00', 'F40.8'

In [5]:
scz_mrns = list(set(all_scz_diagnoses.PrimaryMrn))

In [6]:
controls_depression = pd.read_csv('/Users/valena17/DataGripProjects/msdw2/output/matched_controls_depression.csv')
controls_depression['PrimaryMrn'].nunique()

/var/folders/xm/m4dy5bvs1fn7fb9wxjhsg4dc0000gn/T/ipykernel_41533/3404971775.py:1: DtypeWarning: Columns (0,10,13) have mixed types. Specify dtype option on import or set low_memory=False.
  controls_depression = pd.read_csv('/Users/valena17/DataGripProjects/msdw2/output/matched_controls_depression.csv')


140948

In [9]:
controls_depression = controls_depression[~controls_depression.PrimaryMrn.isin(scz_mrns)]
controls_depression = controls_depression.replace(['Unknown', '*Unknown','*Unspecified', 'Choose not to disclose', 'Not recorded on birth certificate','*Not Applicable', 'Indeterminate', 'Uncertain', 'UNKNOWN', 'PATIENT DECLINED'], np.nan)
controls_depression['PrimaryMrn'].nunique()

136919

In [28]:
controls = pd.read_csv('/Users/valena17/DataGripProjects/msdw2/output/matched_controls.csv')
controls = controls[~controls.PrimaryMrn.isin(scz_mrns)]
controls = controls.replace(['Unknown', '*Unknown','*Unspecified', 'Choose not to disclose', 'Not recorded on birth certificate','*Not Applicable', 'Indeterminate', 'Uncertain', 'UNKNOWN', 'PATIENT DECLINED', '*Deleted'], np.nan)
controls["PrimaryMrn"]=controls["PrimaryMrn"].values.astype(str)

/var/folders/xm/m4dy5bvs1fn7fb9wxjhsg4dc0000gn/T/ipykernel_8430/667518559.py:1: DtypeWarning: Columns (0,10,13) have mixed types. Specify dtype option on import or set low_memory=False.
  controls = pd.read_csv('/Users/valena17/DataGripProjects/msdw2/output/matched_controls.csv')


In [29]:
controls = controls.sort_values(by=['PrimaryMrn','VisitDate'], ascending=[True, True])
controls = controls.drop_duplicates(subset=['PrimaryMrn', 'DisplayString'],  keep='first')

In [30]:
ccsr = pd.read_csv("/Users/valena17/DataGripProjects/msdw2/psych_diagnoses/ccsr_fcodes.csv")
ccsr = ccsr.applymap(lambda x: x.replace("'", ""))
ccsr = ccsr[['ICD-10-CM CODE', 'Default CCSR CATEGORY DESCRIPTION OP']]
ccsr_mapping = ccsr.set_index('ICD-10-CM CODE').T.to_dict('list')
ccsr_mapping['F01518']

['Neurocognitive disorders']

In [31]:
def CCSR(row):
    row = row.replace('.', '')
    if row in ccsr_mapping.keys():
        return ccsr_mapping[row][0]

controls['CCSR_code'] = controls['Value'].apply(CCSR)
controls.head(50)

,PrimaryMrn,AgeInYears,BirthDate,PreferredLanguage,Sex,SexAssignedAtBirth,GenderIdentity,Ethnicity,FirstRace,SecondRace,...,PostalCode,EncounterKey,AgeAtVisit,VisitDate,VisitType,VisitType2,Value,DisplayString,PayorFinancialClass,CCSR_code
1407141,1000094,48.0,1974-12-10,English,Male,NaN,NaN,NaN,OTHER,NaN,...,10029,323315612,47.0,2022-11-12,Hospital Encounter,Emergency,F43.24,Adjustment disorder with disturbance of conduct,Commercial/Managed Care,Trauma- and stressor-related disorders
1808696,1000094,48.0,1974-12-10,English,Male,NaN,NaN,NaN,OTHER,NaN,...,10029,323077124,47.0,2022-11-12,Hospital Encounter,Emergency,F43.20,"Adjustment disorder, unspecified",Commercial/Managed Care,Trauma- and stressor-related disorders
1858877,1000176,48.0,1974-12-23,English,Male,NaN,NaN,NaN,NaN,NaN,...,10025,255234277,40.0,2015-05-18,Office Visit,Outpatient,F32.1,"Major depressive disorder, single episode, mod...",NaN,Depressive disorders
1804800,1000224,75.0,1947-09-29,English,Male,Male,Male,NaN,AFRICAN-AMERICAN,NaN,...,10701,3561374,62.0,2010-06-24,Psych Conf Office Visit,Outpatient,F41.1,Generalized anxiety disorder,Self-Pay,Anxiety and fear-related disorders
1837909,1000224,75.0,1947-09-29,English,Male,Male,Male,NaN,AFRICAN-AMERICAN,NaN,...,10701,154293468,68.0,2016-06-21,Office Visit,Outpatient,F17.210,"Nicotine dependence, cigarettes, uncomplicated",NaN,Tobacco-related disorders
1392173,1000224,75.0,1947-09-29,English,Male,Male,Male,NaN,AFRICAN-AMERICAN,NaN,...,10701-1908,307082686,74.0,2022-08-24,Office Visit,NaN,F17.200,"Nicotine dependence, unspecified, uncomplicated",WTC,Tobacco-related disorders
1838026,1000417,54.0,1968-05-25,English,Female,Female,Female,NaN,AFRICAN-AMERICAN,NaN,...,10039,3068279,41.0,2010-02-03,Psych Conf Telephone,NaN,F33.1,"Major depressive disorder, recurrent, moderate",Self-Pay,Depressive disorders
1430464,1000417,54.0,1968-05-25,English,Female,Female,Female,NaN,AFRICAN-AMERICAN,NaN,...,10039,2948041,41.0,2010-03-19,Psych Conf Office Visit,Recurring,F40.01,Agoraphobia with panic disorder,Self-Pay,Anxiety and fear-related disorders
1392275,1000417,54.0,1968-05-25,English,Female,Female,Female,NaN,AFRICAN-AMERICAN,NaN,...,10039,22520511,46.0,2014-12-05,Psych Conf Office Visit,Recurring,F41.0,Panic disorder (episodic paroxysmal anxiety),Self-Pay,Anxiety and fear-related disorders
1430476,1000417,54.0,1968-05-25,English,Female,Female,Female,NaN,AFRICAN-AMERICAN,NaN,...,10039,14722359,46.0,2015-01-22,Office Visit,Outpatient,F32.A,"Depression, unspecified",Self-Pay,Depressive disorders


In [32]:
controls = controls.replace(['Unknown', '*Unknown','*Unspecified', 'Choose not to disclose', 'Not recorded on birth certificate','*Not Applicable', 'Indeterminate', 'Uncertain', 'UNKNOWN', 'PATIENT DECLINED', '*Deleted'], np.nan)

def ccsr_substance(x):
    substance_disorders = ['Other specified substance-related disorders',
                           'Alcohol-related disorders',
                           'Tobacco-related disorders',
                           'Stimulant-related disorders',
                           'Cannabis-related disorders',
                           'Opioid-related disorders',
                           'Sedative-related disorders',
                           'Hallucinogen-related disorders',
                           'Inhalant-related disorders']
    if x in substance_disorders:
        return 'Substance-related disorders'
    else:
        return x

controls['CCSR_wSubstance'] = controls['CCSR_code'].apply(ccsr_substance)

In [33]:
controls = controls.drop_duplicates(subset=['PrimaryMrn', 'CCSR_wSubstance'],  keep='first')
controls.head(50)

,PrimaryMrn,AgeInYears,BirthDate,PreferredLanguage,Sex,SexAssignedAtBirth,GenderIdentity,Ethnicity,FirstRace,SecondRace,...,EncounterKey,AgeAtVisit,VisitDate,VisitType,VisitType2,Value,DisplayString,PayorFinancialClass,CCSR_code,CCSR_wSubstance
1407141,1000094,48.0,1974-12-10,English,Male,NaN,NaN,NaN,OTHER,NaN,...,323315612,47.0,2022-11-12,Hospital Encounter,Emergency,F43.24,Adjustment disorder with disturbance of conduct,Commercial/Managed Care,Trauma- and stressor-related disorders,Trauma- and stressor-related disorders
1858877,1000176,48.0,1974-12-23,English,Male,NaN,NaN,NaN,NaN,NaN,...,255234277,40.0,2015-05-18,Office Visit,Outpatient,F32.1,"Major depressive disorder, single episode, mod...",NaN,Depressive disorders,Depressive disorders
1804800,1000224,75.0,1947-09-29,English,Male,Male,Male,NaN,AFRICAN-AMERICAN,NaN,...,3561374,62.0,2010-06-24,Psych Conf Office Visit,Outpatient,F41.1,Generalized anxiety disorder,Self-Pay,Anxiety and fear-related disorders,Anxiety and fear-related disorders
1837909,1000224,75.0,1947-09-29,English,Male,Male,Male,NaN,AFRICAN-AMERICAN,NaN,...,154293468,68.0,2016-06-21,Office Visit,Outpatient,F17.210,"Nicotine dependence, cigarettes, uncomplicated",NaN,Tobacco-related disorders,Substance-related disorders
1838026,1000417,54.0,1968-05-25,English,Female,Female,Female,NaN,AFRICAN-AMERICAN,NaN,...,3068279,41.0,2010-02-03,Psych Conf Telephone,NaN,F33.1,"Major depressive disorder, recurrent, moderate",Self-Pay,Depressive disorders,Depressive disorders
1430464,1000417,54.0,1968-05-25,English,Female,Female,Female,NaN,AFRICAN-AMERICAN,NaN,...,2948041,41.0,2010-03-19,Psych Conf Office Visit,Recurring,F40.01,Agoraphobia with panic disorder,Self-Pay,Anxiety and fear-related disorders,Anxiety and fear-related disorders
1805247,1000445,48.0,1974-05-01,English,Female,Female,Female,NaN,OTHER,NaN,...,56386678,42.0,2017-03-23,Admission Eval,Outpatient,F39,Unspecified mood (affective) disorder,Self-Pay,Other specified and unspecified mood disorders,Other specified and unspecified mood disorders
1392487,1000446,48.0,1974-08-23,English,Female,NaN,NaN,NaN,OTHER,NaN,...,53840418,42.0,2017-01-25,Office Visit,Outpatient,F32.A,"Depression, unspecified",Medicaid Managed Care,Depressive disorders,Depressive disorders
1834540,1000539,65.0,1957-05-02,English,Female,NaN,NaN,NaN,OTHER,NaN,...,13512177,55.0,2012-05-14,Psych Conf Office Visit,Recurring,F19.20,"Other psychoactive substance dependence, uncom...",Self-Pay,Other specified substance-related disorders,Substance-related disorders
1805249,1000898,111.0,1911-09-29,English,Female,NaN,NaN,NaN,OTHER,NaN,...,5766465,95.0,2007-04-17,Office Visit,Outpatient,F32.A,"Depression, unspecified",Self-Pay,Depressive disorders,Depressive disorders


In [36]:
controls['Ethnicity_v2'] = controls['Ethnicity']
controls.Ethnicity_v2 = controls.Ethnicity_v2.replace(['Puerto Rican', 'Dominican', 'Latin American', 'Cuban', 'Ecuadorian',
                                                         'Mexican', 'Colombian','Mexican American', 'Criollo', 'Honduran', 'Panamanian',
                                                         'Chilean', 'Central American','South American', 'Guatemalan', 'Venezuelan',
                                                         'Uruguayan', 'Peruvian'],
                    ['Hispanic or Latino', 'Hispanic or Latino', 'Hispanic or Latino', 'Hispanic or Latino',
                     'Hispanic or Latino', 'Hispanic or Latino', 'Hispanic or Latino', 'Hispanic or Latino',
                     'Hispanic or Latino', 'Hispanic or Latino', 'Hispanic or Latino', 'Hispanic or Latino',
                     'Hispanic or Latino', 'Hispanic or Latino', 'Hispanic or Latino', 'Hispanic or Latino',
                     'Hispanic or Latino', 'Hispanic or Latino'])

In [35]:
controls['Merged_Gender'] = controls['GenderIdentity']
controls['Merged_Gender'] = controls['Merged_Gender'].fillna(controls['Sex'])
controls.Merged_Gender.value_counts(dropna=False)

Female                   357517
Male                     274158
Non-binary                 2307
Transgender Female         2033
Transgender Male           1227
Gender Non-conforming       444
Gender Queer                393
Other                       393
NaN                          98
Name: Merged_Gender, dtype: int64

In [37]:
controls['Race'] = controls['FirstRace']
controls.Race = controls.Race.replace(['AMERICAN INDIAN OR ALASKAN', 'Hispanic/Latino', 'Hispanic', 'AFRICAN-AMERICAN', 'OTHER',
                               'PATIENT DECLINED', 'WHITE', 'ASIAN INDIAN', 'CHINESE', 'KOREAN', 'CHINESE AMERICAN', 'VIETNAMESE',
                               'OTHER: NORTH AFRICAN', 'JAPANESE', 'NATIVE HAWAIIAN', 'OTHER PACIFIC ISLANDER', 'THAI',
                               'PAPUA NEW GUINEAN', 'GUINEAN', 'FILIPINO', 'FILIPINO AMERICAN', 'BANGLADESHI', 'UGANDAN', 'PAKISTANI',
                               'WEST INDIAN', 'TRINIDADIAN', 'JAMAICAN', 'HAITIAN', 'TAIWANESE', 'NIGERIAN', 'ETHIOPIAN',
                               'DOMINICA ISLANDER', 'GRENADIAN', 'MADAGASCAR', 'POLYNESIAN', 'SRI LANKAN', 'INDONESIAN', 'MALIAN',
                               'GHANAIAN', 'BARBADIAN', 'TAHITIAN', 'ST VINCENTIAN', 'TOKELAUAN',
                               'Black or African-American', 'Asian (Pacific Islander)', 'LAOTIAN', 'CONGOLESE', 'Native American',
                               'OTHER: WEST AFRICAN', 'BHUTANESE', 'CAMBODIAN', 'Pacific Islander', 'OKINAWAN', 'NEW HEBRIDES',
                               'KENYAN', 'MARIANA ISLANDER', 'SENEGALESE', 'MARSHALLESE', 'BURMESE', 'GABONIAN', 'MALAYSIAN',
                               'TOGOLESE', 'OTHER: EAST AFRICAN', 'LIBERIAN', 'SOMALIAN', 'SUDANESE', 'HMONG', 'MICRONESIAN',
                               'NEPALESE', 'Black or African American', 'OTHER: SOUTH AFRICAN', 'Other Asian', 'ERITREAN', 'IVORY COASTIAN',
                                       'SINGAPOREAN', 'ZIMBABWEAN', 'YAPESE', 'CAPE VERDIAN', 'SAIPANESE', 'SAMOAN', 'MELANESIAN',
                                       'SIERRA LEONEAN', 'GUAMANIAN', 'CAROLINIAN', 'KOSRAEAN', 'GUAMANIAN OR CHAMORRO', 'CHUUKESE',
                                       'MALDIVIAN', 'POHNPEIAN', 'IWO JIMAN', 'FIJIAN', 'TONGAN', 'CHAMORRO', 'SOLOMON ISLANDER', 'TANZANIAN'],
                  ['American Indian or Alaska Native', 'Hispanic or Latino', 'Hispanic or Latino', 'Black or African-American', 'Other Race', 'Decline to Answer', 'White', 'Asian', 'Asian', 'Asian', 'Asian', 'Asian', 'Black or African-American', 'Asian', 'Native Hawaiian or Other Pacific Islander', 'Native Hawaiian or Other Pacific Islander', 'Asian', 'Native Hawaiian or Other Pacific Islander', 'Native Hawaiian or Other Pacific Islander', 'Native Hawaiian or Other Pacific Islander','Native Hawaiian or Other Pacific Islander', 'Asian', 'Black or African-American', 'Asian', 'Asian', 'Black or African-American', 'Black or African-American', 'Black or African-American', 'Asian', 'Black or African-American', 'Black or African-American', 'Black or African-American', 'Black or African-American', 'Black or African-American', 'Native Hawaiian or Other Pacific Islander', 'Asian', 'Asian', 'Black or African-American', 'Black or African-American', 'Black or African-American', 'Native Hawaiian or Other Pacific Islander', 'Black or African-American', 'Native Hawaiian or Other Pacific Islander',
                   'Black or African-American', 'Native Hawaiian or Other Pacific Islander', 'Asian', 'Black or African-American',
                   'American Indian or Alaska Native', 'Black or African-American', 'Asian', 'Asian',
                   'Native Hawaiian or Other Pacific Islander', 'Asian', 'Native Hawaiian or Other Pacific Islander',
                   'Black or African-American', 'Native Hawaiian or Other Pacific Islander', 'Black or African-American',
                   'Native Hawaiian or Other Pacific Islander', 'Asian', 'Black or African-American', 'Asian',
                   'Black or African-American', 'Black or African-American', 'Black or African-American', 'Black or African-American',
                   'Black or African-American', 'Asian', 'Native Hawaiian or Other Pacific Islander', 'Asian', 'Black or African-American', 'Black or African-American', 'Asian', 'Black or African-American', 'Black or African-American', 'Asian', 'Black or African-American',
                   'Native Hawaiian or Other Pacific Islander', 'Black or African-American', 'Native Hawaiian or Other Pacific Islander',
                   'Native Hawaiian or Other Pacific Islander', 'Native Hawaiian or Other Pacific Islander', 'Black or African-American',
                   'Native Hawaiian or Other Pacific Islander', 'Native Hawaiian or Other Pacific Islander',
                   'Native Hawaiian or Other Pacific Islander', 'Native Hawaiian or Other Pacific Islander',
                   'Native Hawaiian or Other Pacific Islander', 'Asian', 'Native Hawaiian or Other Pacific Islander', 'Asian',
                   'Native Hawaiian or Other Pacific Islander', 'Native Hawaiian or Other Pacific Islander',
                   'Native Hawaiian or Other Pacific Islander', 'Native Hawaiian or Other Pacific Islander', 'Black or African-American'])

In [38]:
def multiracial(x):
    if x.MultiRacial == 1:
        return 'Mixed racial group'
    if x.Ethnicity_v2 == 'Hispanic or Latino' and x.Race == 'Other Race':
        return 'Hispanic or Latino'
    if x.Ethnicity_v2 == 'Hispanic or Latino' and x.Race == 'NaN':
            return 'Hispanic or Latino'
    else:
        return x.Race

# scz_notes = scz_notes.apply(multiracial)
controls['Race_2'] = controls.apply(lambda row: multiracial(row), axis=1)
controls.Race_2.value_counts(dropna=False)

White                                        214106
NaN                                          162816
Other Race                                   135651
Black or African-American                     89256
Asian                                         15238
Mixed racial group                            11599
Hispanic or Latino                             5894
Native Hawaiian or Other Pacific Islander      3301
American Indian or Alaska Native                709
Name: Race_2, dtype: int64

In [39]:
controls

,PrimaryMrn,AgeInYears,BirthDate,PreferredLanguage,Sex,SexAssignedAtBirth,GenderIdentity,Ethnicity,FirstRace,SecondRace,...,VisitType2,Value,DisplayString,PayorFinancialClass,CCSR_code,CCSR_wSubstance,Ethnicity_v2,Merged_Gender,Race,Race_2
1407141,1000094,48.0,1974-12-10,English,Male,NaN,NaN,NaN,OTHER,NaN,...,Emergency,F43.24,Adjustment disorder with disturbance of conduct,Commercial/Managed Care,Trauma- and stressor-related disorders,Trauma- and stressor-related disorders,NaN,Male,Other Race,Other Race
1858877,1000176,48.0,1974-12-23,English,Male,NaN,NaN,NaN,NaN,NaN,...,Outpatient,F32.1,"Major depressive disorder, single episode, mod...",NaN,Depressive disorders,Depressive disorders,NaN,Male,NaN,NaN
1804800,1000224,75.0,1947-09-29,English,Male,Male,Male,NaN,AFRICAN-AMERICAN,NaN,...,Outpatient,F41.1,Generalized anxiety disorder,Self-Pay,Anxiety and fear-related disorders,Anxiety and fear-related disorders,NaN,Male,Black or African-American,Black or African-American
1837909,1000224,75.0,1947-09-29,English,Male,Male,Male,NaN,AFRICAN-AMERICAN,NaN,...,Outpatient,F17.210,"Nicotine dependence, cigarettes, uncomplicated",NaN,Tobacco-related disorders,Substance-related disorders,NaN,Male,Black or African-American,Black or African-American
1838026,1000417,54.0,1968-05-25,English,Female,Female,Female,NaN,AFRICAN-AMERICAN,NaN,...,NaN,F33.1,"Major depressive disorder, recurrent, moderate",Self-Pay,Depressive disorders,Depressive disorders,NaN,Female,Black or African-American,Black or African-American
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4579,nan,14.0,2008-03-18,NaN,Male,NaN,NaN,NaN,NaN,NaN,...,Outpatient,F98.0,Enuresis not due to a substance or known physi...,NaN,Urinary incontinence,Urinary incontinence,NaN,Male,NaN,NaN
1089333,nan,83.0,1939-07-05,NaN,Female,NaN,NaN,NaN,NaN,NaN,...,Outpatient,F51.02,Adjustment insomnia,NaN,Sleep wake disorders,Sleep wake disorders,NaN,Female,NaN,NaN
1405654,nan,105.0,1917-11-30,NaN,Female,NaN,NaN,NaN,NaN,NaN,...,Outpatient,F45.42,Pain disorder with related psychological factors,NaN,Somatic disorders,Somatic disorders,NaN,Female,NaN,NaN
2618,nan,10.0,2012-08-20,NaN,Female,NaN,NaN,NaN,NaN,NaN,...,Outpatient,F80.4,Speech and language development delay due to h...,NaN,Hearing loss,Hearing loss,NaN,Female,NaN,NaN


In [40]:
controls_clean = controls.dropna(subset = ['Race_2', 'Merged_Gender', 'PrimaryMrn'])

In [41]:
controls_clean.value_counts('PayorFinancialClass')

PayorFinancialClass
Self-Pay                   176603
Commercial/Managed Care     67831
Medicaid Managed Care       36785
Medicare MS                 11748
Medicare Managed Care        9721
Medicaid MS                  8093
WTC                          1193
Other Medicaid HMO           1146
Self Pay MS                   827
Child Health Plus             709
Aetna                         691
Self Pay                      592
Medicare                      543
Commercial                    465
Medicaid                      407
Commercial Two                364
Transplant                    358
Medicare HMO Two              248
Blue Cross                    207
Medicaid HMO                  201
Worker's Comp/No Fault        194
Cigna                         190
Other                         170
HMO                           148
PPO                            91
Oxford                         91
HMO Other                      88
Wellcare                       75
Blue Cross Blue Shield      

In [42]:
def insurance_names(row): # WTC,
    if search('dicare|dicaid|WTC|Child', str(row)):
        return 'Public'
    if search('Self', str(row)):
        return 'Self-pay'
    if pd.isna(row):
        return 'NaN'
    else:
        return 'Private'

controls_clean['insurance_status'] = controls_clean['PayorFinancialClass'].apply(insurance_names)

/var/folders/xm/m4dy5bvs1fn7fb9wxjhsg4dc0000gn/T/ipykernel_8430/1445753009.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  controls_clean['insurance_status'] = controls_clean['PayorFinancialClass'].apply(insurance_names)


In [43]:
controls_clean.PostalCode = controls_clean.PostalCode.str[:5]
controls_clean.PostalCode = controls_clean.PostalCode.replace(['99999', '00000'],np.nan)
controls_clean.PostalCode.value_counts(dropna=False)

/var/folders/xm/m4dy5bvs1fn7fb9wxjhsg4dc0000gn/T/ipykernel_8430/2975691963.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  controls_clean.PostalCode = controls_clean.PostalCode.str[:5]
/var/folders/xm/m4dy5bvs1fn7fb9wxjhsg4dc0000gn/T/ipykernel_8430/2975691963.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  controls_clean.PostalCode = controls_clean.PostalCode.replace(['99999', '00000'],np.nan)


10029    35966
10025    23716
10019    12067
10035    11442
10027    10633
         ...  
10411        1
33761        1
93908        1
94128        1
08042        1
Name: PostalCode, Length: 5601, dtype: int64

In [44]:
income = pd.read_csv('/Users/valena17/psych_nlp/data/zipcode_income.csv')
income = income.replace(['-'],np.nan)
income.PostalCode = income.PostalCode.str[-5:]

print(income['2021_Median_Hincome'].isna().sum())

income['2021_Median_Hincome'] = income['2021_Median_Hincome'].replace(['250,000+'],'250000')
income['2021_Median_Hincome'] = income['2021_Median_Hincome'].replace(['2,500-'],'2500')
income['2021_Median_Hincome']=income['2021_Median_Hincome'].values.astype(float)

189


In [45]:
controls_clean = pd.merge(controls_clean,income, on=['PostalCode'], how='left')
controls_clean

,PrimaryMrn,AgeInYears,BirthDate,PreferredLanguage,Sex,SexAssignedAtBirth,GenderIdentity,Ethnicity,FirstRace,SecondRace,...,CCSR_code,CCSR_wSubstance,Ethnicity_v2,Merged_Gender,Race,Race_2,insurance_status,2021_Median_Hincome,2020_Median_Hincome,2019_Median_Hincome
0,1000094,48.0,1974-12-10,English,Male,NaN,NaN,NaN,OTHER,NaN,...,Trauma- and stressor-related disorders,Trauma- and stressor-related disorders,NaN,Male,Other Race,Other Race,Private,33901.0,33801,33720
1,1000224,75.0,1947-09-29,English,Male,Male,Male,NaN,AFRICAN-AMERICAN,NaN,...,Anxiety and fear-related disorders,Anxiety and fear-related disorders,NaN,Male,Black or African-American,Black or African-American,Self-pay,55553.0,87397,75182
2,1000224,75.0,1947-09-29,English,Male,Male,Male,NaN,AFRICAN-AMERICAN,NaN,...,Tobacco-related disorders,Substance-related disorders,NaN,Male,Black or African-American,Black or African-American,NaN,55553.0,87397,75182
3,1000417,54.0,1968-05-25,English,Female,Female,Female,NaN,AFRICAN-AMERICAN,NaN,...,Depressive disorders,Depressive disorders,NaN,Female,Black or African-American,Black or African-American,Self-pay,50164.0,41363,42278
4,1000417,54.0,1968-05-25,English,Female,Female,Female,NaN,AFRICAN-AMERICAN,NaN,...,Anxiety and fear-related disorders,Anxiety and fear-related disorders,NaN,Female,Black or African-American,Black or African-American,Self-pay,50164.0,41363,42278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475697,X400206,122.0,1900-11-06,English,Male,NaN,NaN,NaN,WHITE,NaN,...,Alcohol-related disorders,Substance-related disorders,NaN,Male,White,White,Self-pay,105538.0,101651,103792
475698,X400207,36.0,1986-07-26,English,Male,NaN,NaN,NaN,OTHER,NaN,...,Alcohol-related disorders,Substance-related disorders,NaN,Male,Other Race,Other Race,Self-pay,89027.0,93397,88130
475699,X400208,26.0,1996-12-13,English,Male,NaN,NaN,NaN,AFRICAN-AMERICAN,NaN,...,Alcohol-related disorders,Substance-related disorders,NaN,Male,Black or African-American,Black or African-American,Self-pay,NaN,NaN,NaN
475700,X400209,26.0,1996-11-24,English,Male,NaN,NaN,NaN,OTHER,NaN,...,Alcohol-related disorders,Substance-related disorders,NaN,Male,Other Race,Other Race,Self-pay,105538.0,101651,103792


In [47]:
controls_clean['2021_Median_Hincome'] = controls_clean['2021_Median_Hincome'].replace(['250,000+'],'250000')
controls_clean['2021_Median_Hincome']=controls_clean['2021_Median_Hincome'].values.astype(float)
def income_range(x):
    income = x['2021_Median_Hincome']
    if income < 52000:
        return '<52,000'
    if 52000 <= income <= 65999:
        return '52,000 - 65,999'
    if 66000 <= income <= 87999:
        return '66,000 - 87,999'
    if 88000 <= income :
        return '88,000+'

controls_clean['ZIPINC_QRTL'] = controls_clean.apply(lambda row: income_range(row), axis=1)
controls_clean

,PrimaryMrn,AgeInYears,BirthDate,PreferredLanguage,Sex,SexAssignedAtBirth,GenderIdentity,Ethnicity,FirstRace,SecondRace,...,CCSR_wSubstance,Ethnicity_v2,Merged_Gender,Race,Race_2,insurance_status,2021_Median_Hincome,2020_Median_Hincome,2019_Median_Hincome,ZIPINC_QRTL
0,1000094,48.0,1974-12-10,English,Male,NaN,NaN,NaN,OTHER,NaN,...,Trauma- and stressor-related disorders,NaN,Male,Other Race,Other Race,Private,33901.0,33801,33720,"<52,000"
1,1000224,75.0,1947-09-29,English,Male,Male,Male,NaN,AFRICAN-AMERICAN,NaN,...,Anxiety and fear-related disorders,NaN,Male,Black or African-American,Black or African-American,Self-pay,55553.0,87397,75182,"52,000 - 65,999"
2,1000224,75.0,1947-09-29,English,Male,Male,Male,NaN,AFRICAN-AMERICAN,NaN,...,Substance-related disorders,NaN,Male,Black or African-American,Black or African-American,NaN,55553.0,87397,75182,"52,000 - 65,999"
3,1000417,54.0,1968-05-25,English,Female,Female,Female,NaN,AFRICAN-AMERICAN,NaN,...,Depressive disorders,NaN,Female,Black or African-American,Black or African-American,Self-pay,50164.0,41363,42278,"<52,000"
4,1000417,54.0,1968-05-25,English,Female,Female,Female,NaN,AFRICAN-AMERICAN,NaN,...,Anxiety and fear-related disorders,NaN,Female,Black or African-American,Black or African-American,Self-pay,50164.0,41363,42278,"<52,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475697,X400206,122.0,1900-11-06,English,Male,NaN,NaN,NaN,WHITE,NaN,...,Substance-related disorders,NaN,Male,White,White,Self-pay,105538.0,101651,103792,"88,000+"
475698,X400207,36.0,1986-07-26,English,Male,NaN,NaN,NaN,OTHER,NaN,...,Substance-related disorders,NaN,Male,Other Race,Other Race,Self-pay,89027.0,93397,88130,"88,000+"
475699,X400208,26.0,1996-12-13,English,Male,NaN,NaN,NaN,AFRICAN-AMERICAN,NaN,...,Substance-related disorders,NaN,Male,Black or African-American,Black or African-American,Self-pay,NaN,NaN,NaN,None
475700,X400209,26.0,1996-11-24,English,Male,NaN,NaN,NaN,OTHER,NaN,...,Substance-related disorders,NaN,Male,Other Race,Other Race,Self-pay,105538.0,101651,103792,"88,000+"


In [48]:
matched_controls = controls_clean[['PrimaryMrn', 'AgeInYears', 'BirthDate', 'PreferredLanguage', 'SexualOrientation',
                                   'PostalCode', 'EncounterKey', 'AgeAtVisit', 'VisitDate', 'VisitType', 'VisitType2',
                                   'Value', 'DisplayString', 'CCSR_code', 'CCSR_wSubstance', 'Merged_Gender', 'Race_2',
                                'insurance_status', '2021_Median_Hincome', 'ZIPINC_QRTL']]

In [18]:
def current_age_range(x):
    age = x.AgeInYears
    if age < 18:
        return '<18'
    if 18 <= age <= 24:
        return '18-24'
    if 25 <= age <= 34:
        return '25-34'
    if 35 <= age <= 44:
        return '35-44'
    if 45 <= age <= 54:
        return '45-54'
    if 55 <= age <= 64:
        return '55-64'
    if 65 <= age <= 74:
        return '65-74'
    if 75 <= age <= 84:
        return '75-84'
    if 85 <= age:
        return '85+'


# controls_clean['Age_group'] = controls_clean.apply(lambda row: age_range_at_diagnosis(row), axis=1)

matched_controls['Age_group'] = matched_controls.apply(lambda row: current_age_range(row), axis=1)

In [20]:
matched_controls = matched_controls.dropna(subset = ['Age_group'])

In [26]:
matched_controls = pd.read_csv('/Users/valena17/psych_nlp/data/all_sample_controls.csv')

/var/folders/xm/m4dy5bvs1fn7fb9wxjhsg4dc0000gn/T/ipykernel_8430/1983202160.py:1: DtypeWarning: Columns (0,5,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  matched_controls = pd.read_csv('/Users/valena17/psych_nlp/data/all_sample_controls.csv')


In [5]:
matched_controls = matched_controls[matched_controls['CCSR_wSubstance'].isin(['Depressive disorders',
                                                                            'Substance-related disorders',
                                                                            'Bipolar and related disorders',
                                                                            'Trauma- and stressor-related disorders',
                                                                            'Anxiety and fear-related disorders'])]

In [21]:
print(matched_controls.PrimaryMrn.nunique())
print(matched_controls.DisplayString.value_counts(dropna=False),
      matched_controls.CCSR_code.value_counts(dropna=False),
      matched_controls.CCSR_wSubstance.value_counts(dropna=False),
      matched_controls.Merged_Gender.value_counts(dropna=False),
      matched_controls.Race_2.value_counts(dropna=False),
      matched_controls.Age_group.value_counts(dropna=False),
      matched_controls.insurance_status.value_counts(dropna=False),
      matched_controls.ZIPINC_QRTL.value_counts(dropna=False))

263184
Anxiety disorder, unspecified                                               50943
Generalized anxiety disorder                                                37124
Depression, unspecified                                                     20586
Major depressive disorder, recurrent, moderate                              14568
Adjustment disorder, unspecified                                            12574
                                                                            ...  
Other stimulant dependence with intoxication, uncomplicated                     1
Other psychoactive substance dependence with intoxication, uncomplicated        1
Other stimulant abuse with stimulant-induced anxiety disorder                   1
Cannabis dependence with intoxication with perceptual disturbance               1
Other stimulant abuse with withdrawal                                           1
Name: DisplayString, Length: 385, dtype: int64 Anxiety and fear-related disorders          

In [49]:
pd.DataFrame.to_csv(matched_controls, "/Users/valena17/psych_nlp/data/all_sample_controls.csv",index = False)

In [12]:
matched_controls_depressed = matched_controls[matched_controls['CCSR_wSubstance'].isin(['Depressive disorders'])]
print(matched_controls_depressed.PrimaryMrn.nunique())
print(matched_controls_depressed.DisplayString.value_counts(dropna=False),
      matched_controls_depressed.CCSR_code.value_counts(dropna=False),
      matched_controls_depressed.CCSR_wSubstance.value_counts(dropna=False),
      matched_controls_depressed.Merged_Gender.value_counts(dropna=False),
      matched_controls_depressed.Race_2.value_counts(dropna=False),
      matched_controls_depressed.Age_group.value_counts(dropna=False),
      matched_controls_depressed.insurance_status.value_counts(dropna=False),
      matched_controls_depressed.ZIPINC_QRTL.value_counts(dropna=False))

90588
Depression, unspecified                                                                  20586
Major depressive disorder, recurrent, moderate                                           14568
Major depressive disorder, single episode, unspecified                                    9353
Dysthymic disorder                                                                        8923
Other specified depressive episodes                                                       8372
Major depressive disorder, recurrent, unspecified                                         5577
Major depressive disorder, single episode, moderate                                       4869
Major depressive disorder, recurrent severe without psychotic features                    4795
Major depressive disorder, recurrent, mild                                                3910
Major depressive disorder, single episode, mild                                           2630
Major depressive disorder, single episode, s

In [13]:
matched_controls_substance = matched_controls[matched_controls['CCSR_wSubstance'].isin(['Substance-related disorders'])]
print(matched_controls_substance.PrimaryMrn.nunique())
print(matched_controls_substance.DisplayString.value_counts(dropna=False),
      matched_controls_substance.CCSR_code.value_counts(dropna=False),
      matched_controls_substance.CCSR_wSubstance.value_counts(dropna=False),
      matched_controls_substance.Merged_Gender.value_counts(dropna=False),
      matched_controls_substance.Race_2.value_counts(dropna=False),
      matched_controls_substance.Age_group.value_counts(dropna=False),
      matched_controls_substance.insurance_status.value_counts(dropna=False),
      matched_controls_substance.ZIPINC_QRTL.value_counts(dropna=False))

67575
Alcohol abuse, uncomplicated                                         8883
Nicotine dependence, unspecified, uncomplicated                      8768
Alcohol dependence with intoxication, uncomplicated                  6837
Alcohol use, unspecified with intoxication, uncomplicated            6611
Other psychoactive substance abuse, uncomplicated                    3436
                                                                     ... 
Other stimulant abuse with stimulant-induced anxiety disorder           1
Other stimulant abuse with intoxication delirium                        1
Cannabis dependence with intoxication with perceptual disturbance       1
Opioid abuse with unspecified opioid-induced disorder                   1
Other stimulant abuse with withdrawal                                   1
Name: DisplayString, Length: 276, dtype: int64 Alcohol-related disorders                      32797
Tobacco-related disorders                      11104
Other specified substance-r

In [14]:
matched_controls_bipolar = matched_controls[matched_controls['CCSR_wSubstance'].isin(['Bipolar and related disorders'])]
print(matched_controls_bipolar.PrimaryMrn.nunique())
print(matched_controls_bipolar.DisplayString.value_counts(dropna=False),
      matched_controls_bipolar.CCSR_code.value_counts(dropna=False),
      matched_controls_bipolar.CCSR_wSubstance.value_counts(dropna=False),
      matched_controls_bipolar.Merged_Gender.value_counts(dropna=False),
      matched_controls_bipolar.Race_2.value_counts(dropna=False),
      matched_controls_bipolar.Age_group.value_counts(dropna=False),
      matched_controls_bipolar.insurance_status.value_counts(dropna=False),
      matched_controls_bipolar.ZIPINC_QRTL.value_counts(dropna=False))

13382
Bipolar disorder, unspecified                                                          5137
Bipolar II disorder                                                                    1808
Bipolar disorder, current episode depressed, moderate                                   574
Manic episode, unspecified                                                              570
Bipolar disorder, current episode manic without psychotic features, unspecified         560
Bipolar disorder, current episode depressed, mild or moderate severity, unspecified     493
Other bipolar disorder                                                                  457
Bipolar disorder, current episode manic severe with psychotic features                  442
Bipolar disorder, current episode mixed, unspecified                                    410
Bipolar disorder, current episode depressed, severe, without psychotic features         303
Bipolar disorder, current episode manic without psychotic features, modera

In [15]:
matched_controls_trauma = matched_controls[matched_controls['CCSR_wSubstance'].isin(['Trauma- and stressor-related disorders'])]
print(matched_controls_trauma.PrimaryMrn.nunique())
print(matched_controls_trauma.DisplayString.value_counts(dropna=False),
      matched_controls_trauma.CCSR_code.value_counts(dropna=False),
      matched_controls_trauma.CCSR_wSubstance.value_counts(dropna=False),
      matched_controls_trauma.Merged_Gender.value_counts(dropna=False),
      matched_controls_trauma.Race_2.value_counts(dropna=False),
      matched_controls_trauma.Age_group.value_counts(dropna=False),
      matched_controls_trauma.insurance_status.value_counts(dropna=False),
      matched_controls_trauma.ZIPINC_QRTL.value_counts(dropna=False))

67303
Adjustment disorder, unspecified                                      12574
Adjustment disorder with mixed anxiety and depressed mood             11610
Adjustment disorder with depressed mood                                9866
Post-traumatic stress disorder, unspecified                            9029
Adjustment disorder with anxiety                                       8178
Adjustment disorder with disturbance of conduct                        3993
Adjustment disorder with mixed disturbance of emotions and conduct     3613
Reaction to severe stress, unspecified                                 2099
Acute stress reaction                                                  1885
Adjustment disorder with other symptoms                                1720
Post-traumatic stress disorder, chronic                                1043
Other reactions to severe stress                                        380
Dissociative and conversion disorder, unspecified                       347
Conver

In [16]:
matched_controls_anxiety = matched_controls[matched_controls['CCSR_wSubstance'].isin(['Anxiety and fear-related disorders'])]
print(matched_controls_anxiety.PrimaryMrn.nunique())
print(matched_controls_anxiety.DisplayString.value_counts(dropna=False),
      matched_controls_anxiety.CCSR_code.value_counts(dropna=False),
      matched_controls_anxiety.CCSR_wSubstance.value_counts(dropna=False),
      matched_controls_anxiety.Merged_Gender.value_counts(dropna=False),
      matched_controls_anxiety.Race_2.value_counts(dropna=False),
      matched_controls_anxiety.Age_group.value_counts(dropna=False),
      matched_controls_anxiety.insurance_status.value_counts(dropna=False),
      matched_controls_anxiety.ZIPINC_QRTL.value_counts(dropna=False))

107886
Anxiety disorder, unspecified                            50944
Generalized anxiety disorder                             37125
Other specified anxiety disorders                         7535
Panic disorder (episodic paroxysmal anxiety)              7389
Social phobia, unspecified                                1006
Fear of flying                                             791
Agoraphobia with panic disorder                            735
Anxiety disorder due to known physiological condition      720
Claustrophobia                                             284
Separation anxiety disorder of childhood                   265
Other specified phobia                                     249
Other mixed anxiety disorders                              198
Phobic anxiety disorder, unspecified                       164
Other situational type phobia                              113
Agoraphobia, unspecified                                    87
Selective mutism                                